## Necessary imports

In [1]:
import requests
import pandas as pd
import numpy as np
import shap
# import torch
# import captum

In [18]:
LOCAL_PATH: str = '/kaggle/input/eurecat-b-dataset/B/data/'

def _get_train_df():
    train_df: pd.DataFrame = pd.read_csv('/kaggle/input/eurecat-b-dataset/B/train.csv', index_col=0)
    target_columns: str = list(train_df.columns[1:])  # status of the texture, with error or not
    train_df['path'] = LOCAL_PATH + train_df['image_path']
    return train_df

classes_names = ['Taca oli', 'Carrera', 'Agujero', 'Hilo fino horizontal2',
       'Tensión horizontal', 'Barrado', 'Borrissol', 'Hilo fino horizontal', 'Punto escapado']

In [4]:
train_df = _get_train_df()
train_df

,image_path,Taca oli,Carrera,Agujero,Hilo fino horizontal2,Tensión horizontal,Barrado,Borrissol,Hilo fino horizontal,Punto escapado,path
8140,4aaf06a9-frame0-00-08.95.jpg,0,0,0,1,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/4aaf06a...
11389,19c8888a-frame0-00-15.68.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/19c8888...
1945,aa6dfa7d-frame0-00-22.02.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/aa6dfa7...
7981,5e93a55a-frame0-00-12.03.jpg,0,0,0,1,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/5e93a55...
9359,d6f0df39-frame0-00-03.39.jpg,0,0,0,0,0,0,0,1,0,/kaggle/input/eurecat-b-dataset/B/data/d6f0df3...
...,...,...,...,...,...,...,...,...,...,...,...
1628,c432f000-frame0-00-28.18.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/c432f00...
10614,953598ce-frame0-00-02.88.jpg,0,0,0,0,0,0,0,1,0,/kaggle/input/eurecat-b-dataset/B/data/953598c...
11750,f5649ddc-frame0-00-08.92.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/f5649dd...
7718,fc2e6614-frame0-00-17.31.jpg,0,1,0,1,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/fc2e661...


## API posts

In [9]:
URL: str = 'http://84.88.176.103:10003/predict'

def _perform_post(_image_path):
    try:
        response = requests.post(URL, files={'file': open(_image_path,'rb')})
        data = response.json()['output']
        data = float(data)
    except Exception:
        data = np.nan
    return data
    
# _perform_post(train_df['path'].iloc[28])

In [10]:
# train_df['prediction'] = train_df['path'].apply(lambda x: _perform_post(x))
small_train_df: pd.DataFrame = train_df.iloc[np.random.randint(len(train_df), size=100)].copy()
small_train_df['prediction'] = small_train_df['path'].apply(lambda x: _perform_post(x))

In [15]:
small_train_df = small_train_df.dropna()
small_train_df['prediction'] = np.round(small_train_df['prediction'], 0).astype(int)
# small_train_df

,image_path,Taca oli,Carrera,Agujero,Hilo fino horizontal2,Tensión horizontal,Barrado,Borrissol,Hilo fino horizontal,Punto escapado,path,prediction
3127,6ee748da-frame0-00-40.92.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/6ee748d...,0
2252,6660ec1d-frame0-00-16.06.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/6660ec1...,0
1966,20a35e77-frame0-00-21.69.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/20a35e7...,0
4954,4568830b-frame0-00-14.55.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/4568830...,0
3143,6622af48-frame0-00-40.68.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/6622af4...,0
9761,10030190-frame0-00-04.05.jpg,0,0,0,0,0,0,0,1,0,/kaggle/input/eurecat-b-dataset/B/data/1003019...,1
1212,0c2eb4e9-frame0-00-01.72.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/0c2eb4e...,0
5552,70acb47e-frame0-00-03.42.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/70acb47...,0
4580,66f594b2-frame0-00-21.56.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/66f594b...,0
11687,7fd034f9-frame0-00-09.89.jpg,0,0,0,0,0,0,0,0,0,/kaggle/input/eurecat-b-dataset/B/data/7fd034f...,0


## Model explainability with SHAP

#### Model output: precomputed

In [28]:
from PIL import Image
images_shape = np.asarray(Image.open(small_train_df['path'].iloc[0])).shape
_images_array: np.ndarray = np.zeros((len(small_train_df), *images_shape))
    
for _i in range(_images_array.shape[0]):
    _images_array[_i, :] = np.asarray(Image.open(small_train_df['path'].iloc[_i]))

In [ ]:
im_array = _images_array[1:3]
_idx = np.argwhere(_a)

In [39]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import json
import shap
import tensorflow as tf



def _images_to_predictions(im_array: np.ndarray) -> np.ndarray:
    # TODO: restore the same amount of data than the passed array...!
    return small_train_df['prediction'].values


# define a masker that is used to mask out partitions of the input image, this one uses a blurred background
masker = shap.maskers.Image("inpaint_telea", _images_array[0].shape)
# By default the Partition explainer is used for all  partition explainer
explainer = shap.Explainer(_images_to_predictions, masker, output_names=classes_names)

# here we use max_evals evaluations of the underlying model to estimate the SHAP values
shap_values = explainer(_images_array, max_evals=len(small_train_df), batch_size=5, outputs=shap.Explanation.argsort.flip[:1])
shap.image_plot(shap_values)